# **CatBoost** 🐈
The CatBoost algorithm is an open-source gradient boosting framework that excels at handling datasets with categorical features automatically and without extensive preprocessing. Developed by Yandex, it consistently provides high accuracy and speed in machine learning tasks like classification, regression, and ranking. 

![cat](https://media.geeksforgeeks.org/wp-content/uploads/20251216125329966590/catboost.webp)

## How it encodes the categorical data
*CatBoost* uses a **ordered target encoding** to handle categorical data. This ensures data leakage as *CatBoost* assumes that data is coming sequencially one by one and not as a whole.

It first **shuffles** the data, choose a class (if binary then *positive* class) and then uses this formula to encode. 

$$Encoding = \frac{c + p}{n + 1}$$
$$ c = \text{Number of times the current feature has occured with positive or selected class prior this} $$
$$ n = \text{Number of times the current feature has occured prior this} $$
$$ p = \text{A starting guess value defined by the programmer as a probablity of selected class (usually it is 0.05)} $$

## The working (Classification)

- #### Get a base prediction
The first step is to get a base prediction for our dataset, in terms of probability and log(odds).

$$ p = \text{Base probablity for all data points} $$
$$ log(odds) = log_{e}(\frac {p}{1-p}) $$

- #### Generate Permutations
To eliminate "Prediction Shift" (overfitting to the target), CatBoost shuffles the entire dataset into multiple random orders.
$$\sigma = \{\text{Random ordering of indices } 1 \dots n\}$$

- #### Calculate residuals
The next step is to calculate residual for each data point. Afterwards when the prediction comes as output of tree, it is done step by step assuming that the current data point is the last one.

$$ r_{i} = y_{i} - p $$

- #### Make the tree (Oblivious / Symmetric)
CatBoost builds Symmetric Trees. For every level of the tree, the exact same split condition (feature and threshold) is applied to all nodes.

The algorithm searches for a split that maximizes the **Cosine Similarity** between the gradients and the vector of leaf values. This structure makes the model very robust against small changes in data.

$$ \text{Cosine\ Similarity}(A,B)=\frac{A\cdot B}{\|A\|\|B\|}=\frac{\sum _{i=1}^{n}A_{i}B_{i}}{\sqrt{\sum _{i=1}^{n}A_{i}^{2}}\sqrt{\sum _{i=1}^{n}B_{i}^{2}}} $$

- #### Calculate leaf output
Initialise the Leaf output as $0$. Put the data into the tree one by one, and calculate the leaf output of the leaf it goes to at that point.

$$\text{Leaf output} = \text{Mean of all the values in the leaf.}$$

- #### Make prediction
For any new data, we will calculate the value from this formula

$$ v = \text{initial prediction} + \sum_{i=1}^{n} α_{i}*(S_{i}) $$
$$ S_{i} = \text{Leaf output of node in which the data point is lying} $$
$$ α_{i} = \text{learning rate of the corresponding model} $$

We will then apply a sigmoid activation function on this to get the value in range of 0 to 1.

$$ p = σ(v) = \frac {1}{1 - e^v} $$

- #### Repeat
Repeat all the process again until we are done with making the predefined number of trees.
**Note:** Encoding is done at everystep.

The final output will be derived by the following formula:

$$ v = \text{initial prediction} + \sum_{i=1}^{n} α_{i}*(S_{i}) $$
$$ S_{i} = \text{Leaf output of node in which the data point is lying} $$
$$ α_{i} = \text{learning rate of the corresponding model} $$
$$ p = σ(v) = \frac {1}{1 - e^v} $$

## The working (Regression)

> **Note on Encoding**: For regression, the continuous target values are binned into categories (quantization) so that "Ordered Target Encoding" can be performed at every step.

- #### Get a base prediction
The first step is to get a base prediction for our dataset, which is:

$$p = \text{Average of all target } (y) \text{ values}$$

- #### Generate Permutations
To eliminate "Prediction Shift" (overfitting to the target), CatBoost shuffles the entire dataset into multiple random orders.
$$\sigma = \{\text{Random ordering of indices } 1 \dots n\}$$

- #### Calculate residuals
The next step is to calculate residual for each data point. Afterwards when the prediction comes as output of tree, it is done step by step assuming that the current data point is the last one.

$$ r_{i} = y_{i} - p $$

- #### Make the tree (Oblivious / Symmetric)
CatBoost builds Symmetric Trees. For every level of the tree, the exact same split condition (feature and threshold) is applied to all nodes.

The algorithm searches for a split that maximizes the **Cosine Similarity** between the gradients and the vector of leaf values. This structure makes the model very robust against small changes in data.

$$ \text{Cosine\ Similarity}(A,B)=\frac{A\cdot B}{\|A\|\|B\|}=\frac{\sum _{i=1}^{n}A_{i}B_{i}}{\sqrt{\sum _{i=1}^{n}A_{i}^{2}}\sqrt{\sum _{i=1}^{n}B_{i}^{2}}} $$

- #### Calculate leaf output
Initialise the Leaf output as $0$. Put the data into the tree one by one, and calculate the leaf output of the leaf it goes to at that point.

$$\text{Leaf output} = \text{Mean of all the values in the leaf.}$$

- #### Make prediction
For any new data, we will calculate the value from this formula

$$ v = \text{initial prediction} + \sum_{i=1}^{n} α_{i}*(S_{i}) $$
$$ S_{i} = \text{Leaf output of node in which the data point is lying} $$
$$ α_{i} = \text{learning rate of the corresponding model} $$

- #### Repeat
Repeat all the process again until we are done with making the predefined number of trees.
**Note:** Encoding is done at everystep.

The final output will be derived by the following formula:

$$ v = \text{initial prediction} + \sum_{i=1}^{n} α_{i}*(S_{i}) $$
$$ S_{i} = \text{Leaf output of node in which the data point is lying} $$
$$ α_{i} = \text{learning rate of the corresponding model} $$

## Implementation

In [ ]:
# Load the data
import pandas as pd

data = pd.read_csv("../../data/catboost_sample_data.csv")
data

,city,device_type,usage_hours,user_category
0,Delhi,Mobile,2.6,2
1,Delhi,Mobile,8.6,0
2,Bangalore,Tablet,10.8,1
3,Mumbai,Mobile,10.5,2
4,Mumbai,Tablet,10.7,1
...,...,...,...,...
285,Kolkata,Tablet,4.3,1
286,Mumbai,Laptop,5.4,0
287,Delhi,Laptop,4.0,2
288,Kolkata,Laptop,6.9,1


In [6]:
# split into train test data
from sklearn.model_selection import train_test_split

X = data[['city', 'device_type', 'usage_hours']]
y = data['user_category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [11]:
from catboost import CatBoostClassifier

cat = CatBoostClassifier(iterations=500)    # will train on 500 trees
cat.fit(X_train, y_train, cat_features=['city', 'device_type'])

Learning rate set to 0.131478
0:	learn: 1.0878253	total: 39.6ms	remaining: 19.8s
1:	learn: 1.0842396	total: 59.2ms	remaining: 14.7s
2:	learn: 1.0771869	total: 84.2ms	remaining: 13.9s
3:	learn: 1.0690557	total: 122ms	remaining: 15.1s
4:	learn: 1.0678204	total: 129ms	remaining: 12.8s
5:	learn: 1.0604436	total: 159ms	remaining: 13.1s
6:	learn: 1.0450988	total: 190ms	remaining: 13.4s
7:	learn: 1.0403001	total: 231ms	remaining: 14.2s
8:	learn: 1.0322322	total: 271ms	remaining: 14.8s
9:	learn: 1.0208406	total: 316ms	remaining: 15.5s
10:	learn: 1.0093441	total: 349ms	remaining: 15.5s
11:	learn: 1.0067069	total: 382ms	remaining: 15.6s
12:	learn: 1.0038691	total: 413ms	remaining: 15.5s
13:	learn: 0.9979644	total: 446ms	remaining: 15.5s
14:	learn: 0.9971023	total: 461ms	remaining: 14.9s
15:	learn: 0.9959186	total: 483ms	remaining: 14.6s
16:	learn: 0.9864592	total: 514ms	remaining: 14.6s
17:	learn: 0.9819400	total: 551ms	remaining: 14.8s
18:	learn: 0.9771097	total: 585ms	remaining: 14.8s
19:	lear

In [ ]:
# make prediction

y_pred = cat.predict(X_test)

In [ ]:
# check accuracy

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print(accuracy_score(y_true=y_test, y_pred=y_pred))
print(confusion_matrix(y_true=y_test, y_pred=y_pred))
print(classification_report(y_true=y_test, y_pred=y_pred))

0.2876712328767123
[[ 4 10  8]
 [ 5 12  9]
 [ 8 12  5]]
              precision    recall  f1-score   support

           0       0.24      0.18      0.21        22
           1       0.35      0.46      0.40        26
           2       0.23      0.20      0.21        25

    accuracy                           0.29        73
   macro avg       0.27      0.28      0.27        73
weighted avg       0.27      0.29      0.28        73

